In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
# font_location = '/usr/share/fonts/truetype/nanum/NanumGothicOTF.ttf'
font_location = r'C:\Users\Jo\PYDATAexam\font\malgun.ttf' # For Windows
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)

In [ ]:
ps_df = pd.read_excel("data/서울시 공무원 수.xls")
ps_df.columns = ['자치구','공무원수']
ps_df.sort_values('자치구',inplace=True)
ps_df.head()

In [ ]:
age_df = pd.read_csv("data/서울시 연령대 전처리 데이터.csv")
age_df.drop('Unnamed: 0',axis=1,inplace=True)
age_df.head()

In [ ]:
money_df = pd.read_excel("data/서울시 예산결산총괄 통계.xls")
money_df.sort_values('자치구',inplace=True)
money_df.head()

In [ ]:
car_df = pd.read_excel("data/서울 열린데이터 광장/서울시 구별 자동차 등록통계.xls")
car_df.sort_values('자치구',inplace=True)
car_df = car_df[['자치구','소계']]
car_df.columns = ['자치구','자동차수']
car_df.head()

In [ ]:
purpose_df = pd.read_excel("data/서울 열린데이터 광장/서울시 용도지역 현황 통계.xls")
purpose_df.sort_values('자치구',inplace=True)
purpose_df.head()

In [ ]:
life_df = pd.read_csv("data/서울 열린데이터 광장/서울시 생활인구수.csv")
life_df.drop('Unnamed: 0',axis=1,inplace=True)
life_df.head()

In [ ]:
car_df = pd.read_excel("data/서울 열린데이터 광장/서울시 구별 자동차 등록통계.xls")
car_df.sort_values('자치구',inplace=True)
car_df = car_df[['자치구','소계']]
car_df.columns = ['자치구','자동차수']
car_df.head()

In [ ]:
illegal_df = pd.read_excel("data/자치구별 적발건수.xlsx")
illegal_df.sort_values('자치구',inplace=True)
illegal_df.head()

In [ ]:
ride_df = pd.read_csv("data/동별/자치구별/서울시 자치구별 대중교통 총 승차 승객수.csv")
ride_df = ride_df[['자치구','승객수']]
ride_df.columns = ['자치구','대중교통승객수']
ride_df.reset_index(inplace=True)
ride_df.drop(['index'],axis=1,inplace=True)
ride_df.sort_values(by='자치구',inplace=True)
ride_df.head()

In [ ]:
df_all = pd.concat([ps_df['공무원수'],age_df[['30대미만','50대미만','50대이상']],money_df['예산현액'],car_df['자동차수'],life_df['생활인구수'],illegal_df['적발 건수'],ride_df['대중교통승객수']],axis=1)
df_all

In [ ]:
df_all.to_csv("all_variance(3안).csv")

In [ ]:
#스케일링

def standard_scaling(df,scale_columns):
    for col in scale_columns:
        series_mean = df[col].mean()
        series_std = df[col].std()
        df[col] = df[col].apply(lambda x: (x - series_mean)/series_std)
    return df

scale_columns = df_all.columns.tolist()
ndf_all = standard_scaling(df_all,scale_columns)

In [ ]:
ndf_all.head()

In [ ]:
ndf_all.corr()

In [ ]:
ndf_all.to_csv("all_variance(3안).csv")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
colormap = plt.cm.Reds
mask=np.zeros_like(df_all.corr(),dtype=np.bool)
mask[np.triu_indices_from(mask)]=True
plt.figure(figsize=(20, 15))
sns.heatmap(df_all.corr(), linewidths = 0.1, vmax = 1.0,
           square = True,mask=mask, cmap = colormap, linecolor = "white", annot = True, annot_kws = {"size" : 16})

In [ ]:
#다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_df = df_all.drop('적발 건수',axis=1)
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(vif_df.values, i) for i in range(vif_df.shape[1])]

In [ ]:
vif.index = vif_df.columns
vif

In [ ]:
res_df = ndf_all[['50대이상','주거지역','생활인구수']]
res_df.columns = ['50대이상_점수','주거지역_점수','생활인구수_점수']

In [ ]:
res_df['자치구'] = ride_df['자치구']
res_df.set_index('자치구',inplace=True)

In [ ]:
res_df

In [ ]:
res_df['최종점수'] = -1 * 3.507 * 0.00000000000000016 - 8.719 * 0.1 * res_df['50대이상_점수'] - 3.499 * 0.1 * res_df['주거지역_점수'] + 8.447 * 0.1 * res_df['생활인구수_점수']
res_df.sort_values('최종점수',ascending=False)